In [ ]:
# import 

import pandas as pd 

# change setting - to show all the columns 
pd.set_option('display.max_columns', None)

In [ ]:
# import csv
listings = pd.read_csv("/Users/tristakuo/neuefische/capstone-project/data/listings.csv.gz")

# subset: listing 
listing = listings[['id','name', 'description','neighborhood_overview', 'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_minimum_nights', 'minimum_maximum_nights',   
'availability_365','host_id','host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'number_of_reviews','number_of_reviews_ltm', 'first_review','last_review', 'review_scores_rating', 'review_scores_accuracy','review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'license', 'instant_bookable', 'calculated_host_listings_count','reviews_per_month']]

In [ ]:
# check how many null values in each columns
listing.isna().sum()

# null value investigation summary:  
# description is the first part of the intro - does not affect our analysis if null
# neighborhood_overview is located at the lower part of the page "where you will be" - does not affect our analysis if null
# 18 bathrooms_text is null cuz the host did not explicitly choose to show it - can be manually checked 
# bedrooms is null if the host did not explicitly choose it - 121 out of 1370 has a keyword "studio" in the "description" (can be 0 bedrooms? if not specified)
# 339 beds are null 
# 8393 host response time/rate are null and 7405 host acceptance rate are null - info hidden 
# 10 hosts hid info on superhost (double checked with the data where host_id > 2, confirmed these 10 only have 1 listing) - can assign values to listings_count  & total_listing
# 2883 listings have 0 reviews (first_review, last_review, review_scores_rating are null)
# when the review was generated automatically due to a cancellation by the host, counted as 1 review without details on accuracy, communication etc.    
# giving values to all category isn't mandatory for the guests - we see inconsistent null values across all categories 
# 11433 hosts do not reveal their registration info 

In [ ]:
# further narrow down the dataframe 

listing_property = listing[['id','name', 'description','neighborhood_overview', 'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms_text','bedrooms', 'beds', 'amenities', 'price', 'minimum_minimum_nights', 'minimum_maximum_nights',   
'availability_365','host_id','review_scores_rating']]

# check null values
listing_property.isna().sum()

In [ ]:
# change neighbourhood_group_cleansed column name
listing_property.rename(columns= {'neighbourhood_group_cleansed':'neighborhood'}, inplace= True)

In [ ]:
# change data types

# id 
listing_property['id'] = listing_property['id'].astype(object)

# price in remove $ sign, remove "." 
listing_property['price'] = listing_property['price'].str.replace('$','')
listing_property['price'] = listing_property['price'].str.replace('.', '')
listing_property['price'] = listing_property['price'].str.replace(',', '')
listing_property['price'] = listing_property['price'].astype(float)

# clean price / 100 
listing_property['price'] = listing_property['price']/100 

# host_id
listing_property['host_id'] = listing_property['host_id'].astype(object)

In [ ]:
# check duplicates - no duplicates!

listing_property.duplicated(subset='id').sum()

In [ ]:
listing_property.duplicated().value_counts()

In [ ]:
#outliers checks
  
listing_property.describe()


In [ ]:
# drop price = 0
listing_property =  listing_property.drop(listing[listing['price'] == 0].index)


In [ ]:
# drop null values for 'bathrooms_text', 'bedrooms', 'beds'
listing_property.dropna(axis= 0, how= 'any', subset= ['bathrooms_text', 'bedrooms', 'beds'], inplace= True)
listing_property.isna().sum()


In [ ]:
# double check null values
listing_property.isna().sum()

In [ ]:
# check categorical data 
listing_property['neighborhood'].value_counts()

In [ ]:
listing_property['property_type'].value_counts()

In [ ]:
listing_property['room_type'].value_counts()

In [ ]:
listing_property['accommodates'].value_counts()

In [ ]:
listing_property['bathrooms_text'].value_counts()

In [ ]:
listing_property['bedrooms'].value_counts()

In [ ]:
listing_property['beds'].value_counts()

In [ ]:
# create new column for bathroom type
# return 1/2 in a new column if bathrooms are shared  
import numpy as np

In [ ]:
listing_property['bathrooms_text'] = listing_property['bathrooms_text'].astype(str)
listing_property['bathrooms_text'] = listing_property['bathrooms_text'].str.lower()

In [ ]:
listing_property['share_bath'] = listing_property['bathrooms_text'].apply(lambda x: np.select([x.__contains__('shared')], '1'))

In [ ]:
listing_property['share_bath'].value_counts()

In [ ]:
# store dataframes for other ipynbs
%store listing
%store listing_property
%store calendar

In [ ]:
# get tables to the database
# import packages
import sql_functions as sf
import sqlalchemy 
from sql_functions import get_sql_config
from sql_functions import get_engine
import psycopg2

sql_config = get_sql_config()
engine = get_engine()


In [ ]:
# upload listing

table_name = 'listing'
schema = 'capstone_tk' 


if engine!=None:
    try:
        listing.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

In [ ]:
# upload listing_property

table_name = 'listing_property'
schema = 'capstone_tk' 


if engine!=None:
    try:
        listing_property.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

In [ ]:
# upload calendar

table_name = 'calendar'
schema = 'capstone_tk' 


if engine!=None:
    try:
        calendar.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

In [ ]:
%store -r listing_property_t

In [ ]:
# upload listing_property_t

table_name = 'listing_property_t'
schema = 'capstone_tk' 


if engine!=None:
    try:
        listing_property_t.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None